In [48]:
import numpy as np
import math
from scipy import optimize

def f(x):
    a = x[0]**4+(x[1]-3)**2
    b = 100-x[0]**4+x[1]**2
    return (a/b)
    # return (x[0]**4+(x[1]-3)**2)/(100+x[0]**4+x[1]**2)


def grad(x):
    return np.array([4*x[0]**3*(x[0]**4 + (x[1] - 3)**2)/(-x[0]**4 + x[1]**2 + 100)**2 + 4*x[0]**3/(-x[0]**4 + x[1]**2 + 100), -2*x[1]*(x[0]**4 + (x[1] - 3)**2)/(-x[0]**4 + x[1]**2 + 100)**2 + (2*x[1] - 6)/(-x[0]**4 + x[1]**2 + 100)])


def hessiano(x):
    return [[4*x[0]**2*(8*x[0]**4/(-x[0]**4 + x[1]**2 + 100) + (x[0]**4 + (x[1] - 3)**2)*(8*x[0]**4/(-x[0]**4 + x[1]**2 + 100) + 3)/(-x[0]**4 + x[1]**2 + 100) + 3)/(-x[0]**4 + x[1]**2 + 100), 8*x[0]**3*(-2*x[1]*(x[0]**4 + (x[1] - 3)**2)/(-x[0]**4 + x[1]**2 + 100) - 3)/(-x[0]**4 + x[1]**2 + 100)**2], [8*x[0]**3*(-2*x[1]*(x[0]**4 + (x[1] - 3)**2)/(-x[0]**4 + x[1]**2 + 100) - 3)/(-x[0]**4 + x[1]**2 + 100)**2, 2*(-4*x[1]*(x[1] - 3)/(-x[0]**4 + x[1]**2 + 100) + (x[0]**4 + (x[1] - 3)**2)*(4*x[1]**2/(-x[0]**4 + x[1]**2 + 100) - 1)/(-x[0]**4 + x[1]**2 + 100) + 1)/(-x[0]**4 + x[1]**2 + 100)]]


In [49]:
Nfeval = 1 
def callbackF(Xi):
    global Nfeval
    print(Nfeval, Xi, f(Xi))
    Nfeval += 1

x0 = np.array([1,2])
optimize.minimize(f, x0, method='BFGS', callback=callbackF, jac=grad,
                  options={"disp": True})


1 [0.80205486 2.10085776] 0.01175273035955997
2 [0.52609669 2.25958707] 0.005948989904260008
3 [0.2972955  2.40635704] 0.003405317332807616
4 [0.0098271  2.60159355] 0.0014866559128864324
5 [-0.3946372   2.87947543] 0.00035819423403072453
6 [-0.34067587  2.84692575] 0.00034139275939130243
7 [-0.34989273  2.85742359] 0.00032654584036805436
8 [-0.35577995  2.87223989] 0.0002988438158605732
9 [-0.35634281  2.89617881] 0.00024824564107763284
10 [-0.34345834  2.9379996 ] 0.00016350402446558193
11 [-0.29114041  3.01249067] 6.730429115674264e-05
12 [-0.25668735  3.02000519] 4.345361423229815e-05
13 [-0.18723489  3.01154123] 1.2489315541621813e-05
14 [-0.14244677  3.00164733] 3.8018871824940733e-06
15 [-0.10942445  2.99833011] 1.3410262013427456e-06
16 [-0.08353154  2.99857921] 4.652146158604252e-07
17 [-0.06272006  2.99961581] 1.433280652957219e-07
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 17
         Function evaluations: 19
        

      fun: 1.433280652957219e-07
 hess_inv: array([[969.44096864,   2.86355329],
       [  2.86355329,  37.37385652]])
      jac: array([-9.05445195e-06, -7.05732164e-06])
  message: 'Optimization terminated successfully.'
     nfev: 19
      nit: 17
     njev: 19
   status: 0
  success: True
        x: array([-0.06272006,  2.99961581])

In [50]:
xstar = np.array([-0.06174568,  2.99969135])
print(grad(xstar))
print(np.linalg.det(hessiano(xstar)))



[-8.63894525e-06 -5.67078781e-06]
7.70195879100483e-06


In [51]:
#Calculos de phi
def phiAlpha(x0, alpha, p):
    paX = x0 + p * alpha
    return f(paX)


def phipAlpha(x0, alpha, p):
    x = x0 + alpha * p
    vgrad = grad(x)
    return (np.dot(vgrad, p))


def phipp(x0, alpha, p):
    x = x0 + alpha * p
    ahess = hessiano(x)
    return np.dot(np.dot(ahess, p), p)


# Gradiente Conjugado NL
def newton(xo, p, ao, itmax=100, tol=1e-12):
    k = 0
    # ak = 0
    while abs(phipAlpha(xo, ao, p)) > tol:
        phiap = phipAlpha(xo, ao, p)
        phiapp = phipp(xo, ao, p)
        ak = ao - phiap/phiapp
        # print(f"\t {k}, {ak}")
        k = k+1
        ao = ak
        if k >= itmax:
            print("Iteraciones exedidas")
            break
    return ak


def gradientesConjugados(x0, flavor="FR", k=0, tol=1e-6):
    beta = 0
    p = -grad(x0)
    print("k, fx, x0, alpha, beta, ")
    while np.linalg.norm(grad(x0)) >= tol:
        alpha = newton(x0, p, 0)
        xi = x0 + alpha*p
        gradxi = grad(xi)
        gradx0 = grad(x0)
        if flavor == "FR":
            beta = np.dot(gradxi, gradxi)/np.dot(gradx0, gradx0)
        elif flavor == "PR":
            beta = np.dot(gradxi, (gradxi - gradx0)) / \
                (np.linalg.norm(gradx0))**2
        elif flavor == "PR+":
            beta = np.dot(gradxi, (gradxi - gradx0)) / \
                (np.linalg.norm(gradx0))**2
            if beta < 0:
                beta = 0
        elif flavor == "HS":
            beta = np.dot(gradxi, (gradxi - gradx0)) / \
                np.dot((gradxi - gradx0), p)
        p = -gradxi + beta*p
        print(k,
              #  f(x0),
              xi, alpha, beta)
        x0 = xi
        k += 1
    return x0


In [52]:
gradientesConjugados(x0,"PR+")

k, fx, x0, alpha, beta, 
0 [-0.4153612   2.72116023] 35.75134982794711 0.017549317315173493
1 [-0.30938852  3.02118856] 53.59077195496832 0.012577679852078569
2 [-0.14977092  2.97619336] 143.72467202199738 0.18172145823329153
3 [-0.12649303  3.0034627 ] 71.56578370343873 0.13585450009186137
4 [-0.0555719   2.99650371] 598.7513363081988 0.8138791760426157
5 [-0.04821912  3.00042102] 71.59343198184696 0.1313610424916864
6 [-0.01965618  2.99955358] 1622.416729981998 1.6887889055093748
7 [-0.01763992  3.00004328] 67.18601426861495 0.10596703076113778


array([-0.01763992,  3.00004328])

In [53]:
def newtonNDim(x0, tol=1e-4):
    k = 0
    while np.linalg.norm(grad(x0)) > tol:
        x0 = x0 - np.dot(np.linalg.inv(hessiano(x0)), grad(x0))
        k = k+1
        print(k, f(x0), x0)
    return x0
newtonNDim(x0)

1 0.0021563386249199403 [0.69306542 2.94429407]
2 0.00042563575337727154 [0.46402636 2.99763088]
3 8.423869375752761e-05 [0.30954279 2.99957428]
4 1.6646141414261858e-05 [0.20638697 2.99991576]
5 3.288376603285467e-06 [0.13759463 2.99998335]


array([0.13759463, 2.99998335])

In [54]:
def levenbergMarquardt(x0, tol=1e-6):
    k = 0
    mu = 1e-3
    v = 10
    n = np.shape(hessiano(x0))
    I = np.identity(n[0])
    print(k, x0, f(x0))
    while np.linalg.norm(grad(x0)) >= tol:
        mu = mu/v
        while True:
            delta = np.dot(np.linalg.inv(hessiano(x0) + mu*I), grad(x0))
            x1 = x0 - delta
            mu = mu * v
            # k = k+1
            print(k,  x0, round(f(x0), 6), x1, round(f(x1), 6), mu)
            if (f(x1) < f(x0)):
                x0 = x1
                break
        k = k+1
    return x0

levenbergMarquardt(x0)

0 [1 2] 0.019417475728155338
0 [1 2] 0.019417 [0.69323069 2.93975236] 0.002164 0.001
1 [0.69323069 2.93975236] 0.002164 [0.46461871 2.99729184] 0.000428 0.001
2 [0.46461871 2.99729184] 0.000428 [0.3105884  2.99956671] 8.5e-05 0.001
3 [0.3105884  2.99956671] 8.5e-05 [0.20804961 2.9999159 ] 1.7e-05 0.001
4 [0.20804961 2.9999159 ] 1.7e-05 [0.14012839 2.99998385] 4e-06 0.001
5 [0.14012839 2.99998385] 4e-06 [0.09548453 2.99999702] 1e-06 0.001
6 [0.09548453 2.99999702] 1e-06 [0.06654008 2.9999995 ] 0.0 0.001
7 [0.06654008 2.9999995 ] 0.0 [0.04813576 2.99999994] 0.0 0.001
8 [0.04813576 2.99999994] 0.0 [0.03660918 3.00000001] 0.0 0.001
9 [0.03660918 3.00000001] 0.0 [0.02933568 3.00000001] 0.0 0.001


array([0.02933568, 3.00000001])